In [13]:
#instalamos la libreria de stramlit
%pip install streamlit


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
#instalamos la libreria de PLOTLY
%pip install plotly


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
#instalamos la libreria de funpymodeling
%pip install funpymodeling


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
#para instalar npm en visual
#desde google escrbir node.js
#instalar la version mas recomendada
! npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
up to date, audited 23 packages in 3s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠴npm notice
npm notice New minor version of npm available! 10.8.2 -> 10.9.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.9.0
npm notice To update run: npm install -g npm@10.9.0
npm notice
⠴

In [23]:
#Importamos librerias
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from funpymodeling.exploratory import freq_tbl
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [24]:
#Carga desde un archivo .csv sin indice y eliminar filas innecesarias
dft=pd.read_csv("mexico_limpio.csv")
dft.head(5)

,Unnamed: 0,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,...,room_type,bathrooms_text,amenities,calendar_updated,has_availability,calendar_last_scraped,first_review,last_review,license,instant_bookable
0,0,3.0,6.0,19.42063,-99.16586,2.0,1.0,1.0,2.0,1652.0,...,Entire home/apt,1 bath,"[""40 inch HDTV with Amazon Prime Video, Netfli...",nulos,t,2024-06-28,2013-03-30,2024-04-17,nulos,f
1,1,1.0,1.0,19.38283,-99.20000,2.0,1.0,1.0,1.0,3663.0,...,Entire home/apt,1 bath,"[""Hot water"", ""Washer"", ""Bed linens"", ""Wifi"", ...",nulos,t,2024-06-27,2013-03-30,2024-04-17,nulos,f
2,2,17.0,17.0,19.41162,-99.17794,3.1,1.4,5.0,1.8,1314.4,...,Entire home/apt,5.5 baths,"[""Wifi"", ""Coffee maker: Nespresso"", ""Host gree...",nulos,t,2024-06-28,2011-11-09,2023-03-26,nulos,f
3,3,1.0,4.0,19.43956,-99.17263,4.0,1.0,2.0,3.0,814.0,...,Entire home/apt,1 bath,"[""Wifi"", ""Host greets you"", ""Coffee maker"", ""S...",nulos,t,2024-06-28,2013-05-16,2024-05-21,nulos,t
4,4,3.0,4.0,19.41152,-99.16857,4.0,1.0,2.0,2.0,1923.0,...,Entire home/apt,1 bath,"[""Wifi"", ""Coffee maker"", ""Sound system"", ""Clot...",nulos,t,2024-06-28,2011-11-17,2024-05-05,nulos,f


In [77]:
%%writefile app.py 
# Creamos el archivo de la APP en el interprete principal (Python)

# Importamos librerías
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from funpymodeling.exploratory import freq_tbl
from sklearn.linear_model import LinearRegression
from PIL import Image

# Definimos la instancia de streamlit
@st.cache_resource
def load_data():
    df = pd.read_csv("mexico_limp.csv", encoding='utf-8', index_col=0)
    df2= pd.read_csv("mexico_limp.csv", index_col="name")
    return df

# Cargamos los datos
data = load_data()
categoric_cols = data.select_dtypes(include=['object']).columns.tolist()  # Obtener columnas categóricas
numeric_cols = data.select_dtypes(include=['float', 'int']).columns.tolist()  # Obtener columnas numéricas

# Función para mostrar imágenes
def show_airbnb_image():
    img = Image.open("airbnb_logo.png")  # Asegúrate de tener esta imagen en el directorio
    st.image(img, caption='Airbnb Logo', use_column_width=True)

# Página principal
st.title('AIRBNB - ANÁLISIS DE DATOS')
st.header('Patricio Mijares González | A01737684') 

# Sidebar - Selección de Página
# Insertar la imagen en la barra lateral (sidebar)
# URL of the image
image = "https://lauranoesta.com/wp-content/uploads/2019/09/airbnb-logo.png"  
st.sidebar.image(image, use_column_width=True)   
st.sidebar.title('MENU')
st.sidebar.header('Seleccione una Sección')
section = st.sidebar.selectbox(label='Sección', options=['Dataset', 'Extracción de Características', 'Regresión Lineal Simple', 'Regresión Lineal Múltiple', 'Regresión No Lineal', 'Regresión Logística'])

# CONTENIDO DEL Dataset 
if section == "Dataset":
    # Generamos los encabezados para el dashboard
    st.header("MÉXICO")
    
    # Widget 2: Checkbox
    # Generamos un cuadro de selección (Checkbox) en una barra lateral (sidebar) para mostrar dataset
    check_box = st.sidebar.checkbox(label="Mostrar Dataset")
    
    # Condicional para que aparezca el dataframe
    if check_box:
        # Mostramos el dataset
        st.write(data)  # Asegúrate de que 'data' esté definido

    ###################
    # Extracción de características 
    extraccion = st.selectbox(label="Extracción de características", options=["Line Plot", "Bar Plot", "Box Plot"])

    # Widget 3: Multiselect box
    if extraccion == 'Line Plot':
        st.subheader('Line Plot')
        st.sidebar.subheader('Tipo de Variables')
        Vars_num_selected = st.sidebar.multiselect(label="Variables graficadas", options=numeric_cols)  # Asegúrate de usar 'numeric_cols'

        # Despliegue de un line plot, definiendo las variables "X categorías" y "Y numéricas" 
        if Vars_num_selected:
            figure1 = px.line(data_frame=data, x=data.index, 
                              y=Vars_num_selected, title='Mexico Line Plot', 
                              width=1600, height=600)
            # Mostramos el lineplot
            st.plotly_chart(figure1)
    
    # Bar plot
    elif extraccion == 'Bar Plot':
        st.subheader('Bar Plot')
        
        # Menú desplegable de opciones de las variables numéricas graficadas
        Vars_Num = st.selectbox(label="Variables", options=["bathrooms", "bethrooms", "accommodates"])
        
        # Asegúrate de que Filtro_index1 esté definido y que sea un DataFrame
        if Vars_Num == "bathrooms":
            y_value = "bathrooms"  # Cambia esto según tu lógica
        elif Vars_Num == "bethrooms":
            y_value = "bethrooms"  # Cambia esto según tu lógica
        elif Vars_Num == "accommodates":
            y_value = "accommodates"  # Cambia esto según tu lógica

        # Despliegue de un bar plot
        figure2 = px.bar(data_frame=data, x=data.index, 
                         y=y_value, title='Análisis Univariado')
        figure2.update_xaxes(automargin=True)
        figure2.update_yaxes(automargin=True)

        # Widget 4: Button
        Button = st.sidebar.button(label="Mostrar gráfica tipo BAR PLOT")
        if Button:
            # Mostramos el barplot
            st.plotly_chart(figure2)
    
    # Boxplot  
    elif extraccion == "Box Plot":
        st.subheader("Box Plot")
        
        # Seleccionar la columna numérica para el boxplot
        columna_numerica = st.selectbox(label="Selecciona una columna numérica:", options=numeric_cols)

        # Crear el boxplot con Plotly Express
        fig = px.box(data, y=columna_numerica)  # Asegúrate de usar 'data'

        # Mostrar el gráfico en Streamlit
        st.plotly_chart(fig)

# CONTENIDO - Extracción de Características
if section == 'Extracción de Características':
    st.subheader('Extracción de Características')
    
    st.sidebar.subheader('Tipo de Variables')
    vartype = st.sidebar.selectbox(label= 'Opción categorica', options=['Categorica'])

    if vartype == 'Categorica':
        st.text('Seleccione la variable y el tipo de gráfico')

        selected_var = st.selectbox(label='Variable', options=categoric_cols)
        selected_graph = st.selectbox(label='Gráfico', options=['Line Plot', 'Bar Plot', 'Scatter Plot', 'Pie Plot'])

        # Análisis univariado para variable categórica
        table = freq_tbl(data[selected_var])
        table1 = table.drop(['percentage', 'cumulative_perc'], axis=1)
        filtro = table1[table1['frequency'] > 1]
        filtro_index = filtro.set_index(str(selected_var))

        # Checkbox para mostrar tabla de frecuencias
        check_box = st.sidebar.checkbox(label="Mostrar Tabla de Frecuencias")
        if check_box:
            st.write(filtro_index)

        # Gráficos
        if selected_graph == 'Line Plot':
            figureEC = px.line(filtro_index, x=filtro_index.index, y='frequency', title='Line Plot', width=1600, height=600)
            st.plotly_chart(figureEC)

        elif selected_graph == 'Bar Plot':
            figureEC = px.bar(filtro_index, x=filtro_index.index, y='frequency', title='Bar Plot')
            st.plotly_chart(figureEC)

        elif selected_graph == 'Scatter Plot':
            figureEC = px.scatter(filtro_index, x=filtro_index.index, y='frequency', title="Scatter Plot")
            st.plotly_chart(figureEC)

        elif selected_graph == 'Pie Plot':
            figureEC = px.pie(filtro_index, names=filtro_index.index, values='frequency', title='Pie Plot', width=1600, height=600)
            st.plotly_chart(figureEC)

# CONTENIDO - Regresión Lineal Simple
if section == 'Regresión Lineal Simple':
    st.subheader('Regresión Lineal Simple')
   
    # Lógica para regresión lineal simple
    check_box = st.sidebar.checkbox(label="Mostrar Dataset")
    if check_box:
        st.write(Filtro_index1)
        st.write(df2)

    x_selected = st.sidebar.selectbox(label="(x) Variable independiente", options=numeric_cols)
    y_selected = st.sidebar.selectbox(label="(y) Variable dependiente", options=numeric_cols)

    # Modelo y resultados
    X = data[x_selected].values.reshape(-1, 1)
    y = data[y_selected].values
    model = LinearRegression()
    model.fit(X, y)
    coef_Deter = model.score(X, y)
    coef_Correl = np.sqrt(coef_Deter)

    # Predecir los valores de y para la línea de tendencia
    y_pred = model.predict(X)

    # Crear gráfico de dispersión con la línea de tendencia
    figure1 = px.scatter(data_frame=data, x=x_selected, y=y_selected,
                         title='Regresión lineal', color_discrete_sequence=["pink"])
    
    # Agregar línea de tendencia
    figure1.add_scatter(x=data[x_selected], y=y_pred, mode='lines', name='Línea de tendencia', line=dict(color='red', width=2))

    # Mostrar gráfico
    st.plotly_chart(figure1)
    
    st.markdown(f"<p style='font-size: 20px; color: white;'> Coeficiente de correlación (R): {coef_Correl:.4f}</p>", unsafe_allow_html=True)


# CONTENIDO - Regresión Lineal Múltiple
if section == 'Regresión Lineal Múltiple':
    st.subheader('Regresión Lineal Múltiple')
    # Lógica para regresión lineal múltiple
    # Seleccionar la variable dependiente (y) y las independientes (x)
    var_dep = st.sidebar.selectbox(label="Variable dependiente (x)", options=numeric_cols)  # Cambia numeric_cols por tus columnas
    vars_indep = st.sidebar.multiselect(label="Variables independientes (y)", options=numeric_cols)

    if vars_indep and var_dep:
        # Declaramos las variables dependientes e independientes para la regresión lineal
        Vars_Indep = data[vars_indep]
        Var_Dep = data[var_dep]

        # Definimos y ajustamos el modelo de regresión lineal
        model = LinearRegression()
        model.fit(X=Vars_Indep, y=Var_Dep)

        # Predecimos los valores usando el modelo ajustado
        y_pred = model.predict(X=Vars_Indep)

        # Insertamos las predicciones en el DataFrame
        data['Predicciones'] = y_pred

        # Visualizamos los gráficos comparativos
        for var in vars_indep:
            # Gráfico de dispersión de la variable independiente vs total real
            figure = px.scatter(data, x=var, y=var_dep, title=f'Relación entre {var} y {var_dep}',
                                labels={var: var, var_dep: 'Total'},
                                color_discrete_sequence=["orange"], opacity=0.5)

            # Agregar las predicciones como puntos
            figure.add_scatter(x=data[var], y=data['Predicciones'], mode='markers', name='Predicciones', 
                               marker=dict(color='purple', size=10, symbol='star', line=dict(width=2, color='white')), opacity=0.8)

            # Mostrar el gráfico
            st.plotly_chart(figure)

        # Coeficiente de determinación (R²) y coeficiente de correlación
        coef_Deter = model.score(Vars_Indep, Var_Dep)
        coef_Correl = np.sqrt(coef_Deter)

        st.markdown(f"<p style='font-size: 20px; color: white;'> Coeficiente de correlación (R): {coef_Correl:.4f}</p>", unsafe_allow_html=True)
        st.markdown(f"<p style='font-size: 20px; color: white;'> Coeficiente de determinación (R²): {coef_Deter:.4f}</p>", unsafe_allow_html=True)
    else:
        st.warning("Selecciona una variable independiente y una variable dependiente.")

     # Definición de las funciones no lineales
def funcion_cuadratica(x, a, b, c):
    return a * x**2 + b * x + c

def funcion_exponencial(x, a, b, c):
    return a * np.exp(b * x) + c

def funcion_senoidal(x, a, b):
    return a * np.sin(x) + b

def funcion_tangencial(x, a, b):
    return a * np.tan(x) + b

def funcion_valor_absoluto(x, a, b, c):
    return a * np.abs(x) + b * x + c

def funcion_cociente_polinomios(x, a, b, c):
    return (a * x**2 + b) / (c * x)

def funcion_logaritmica(x, a, b):
    return a * np.log(x) + b

def funcion_polinomial_inversa(x, a, b, c):
    return a / b * x**2 + c * x

# CONTENIDO - Regresión No Lineal
if section == 'Regresión No Lineal':
    st.subheader('Regresión No Lineal')
    # Lógica para regresión no lineal


# CONTENIDO - Regresión Logística
if section == 'Regresión Logística':
    st.subheader('Regresión Logística')
    # Lógica para regresión logística
    # ...

Overwriting app.py
